In [121]:
import gym
import numpy
env = gym.make('FrozenLake-v0')
new_state = env.reset()


[2017-08-24 15:27:06,836] Making new env: FrozenLake-v0


In [247]:
def q_learn(iterations, q_values):
    total_reward = 0
    for i in range(iterations):
        new_state = env.reset()
        state = new_state 
        for t in range(100):
            if numpy.random.random_sample() > (1 - i/100):
                action = numpy.argmax(q_values[state])
            else:
                action = int(numpy.random.choice(q_values[state]))
            new_state, reward, done, info = env.step(action)
            q_values[state][action] = reward + 0.5*numpy.max(q_values[new_state]) + 0.5*q_values[state][action] - 0.01
            state = new_state
            if done:
                if reward == 0:
                    q_values[state][action] = -1
                else:
                    q_values[state][action] = 1
                    total_reward += 1
                break
    return q_values, total_reward

In [253]:
print (q_learn(1000, numpy.zeros((env.observation_space.n, env.action_space.n))))

(array([[-1.09774162,  0.04472298, -1.09732304, -1.10193512],
       [-1.09719862, -0.92735665, -1.09276472, -1.10103039],
       [-1.07971417, -0.74611722, -1.08634474, -1.08455219],
       [-1.0788538 , -1.03463151, -1.08060425, -1.07938565],
       [-1.06389902, -0.57407805, -1.04524793, -1.05330705],
       [-1.        , -1.        , -1.        , -1.        ],
       [-1.01957831, -0.83261487, -1.02889947, -1.01866562],
       [-1.        , -1.        , -1.        , -1.        ],
       [-1.01941625,  0.09367283, -1.02922186, -1.02      ],
       [-0.71006486,  1.96411261, -0.7017842 , -0.77981027],
       [ 2.05881058, -1.        , -0.7326055 , -0.70979857],
       [ 0.        ,  0.        , -1.        ,  0.        ],
       [-1.        , -1.        , -1.        , -1.        ],
       [-0.0275    , -0.515     ,  2.18891849, -0.02      ],
       [-0.015     , -0.02      , -0.10983497,  2.62782432],
       [ 0.        ,  0.        ,  1.        ,  1.        ]]), 114)
